In [ ]:
import requests
from tqdm import tqdm 
import pandas as pd

import os, json
from pprint import pprint
import statsmodels.api as sm

materials = ['moissanite','birthstone',
             'cubic zirconia',
             'sterling silver',
             # I added these gold-related ones based on the commonly occuring phrases in the titles of in top products 
             "solid gold", "gold plated", 
             # crystals 
             # see https://www.holisticshop.co.uk/crystals/popular-crystals
             'amethyst','aquamarine','lapis lazuli',
            'moonstone','pearl','peridot',
            'jade','citrine','rose quartz','tourmaline',
             'blue lace agate','sunstone','clear quartz',
             'aura quartz','black agate','black jasper',
             'agate','aventurine','calcite','carnelian','fluorite',
             'hematite','jasper','labradorite','malachite','moldavite',
             'obsidian','opalite','selenite','shungite',"tiger's eye",
             'turquoise','orgone','topaz','garnet','lepidolite','smoky quartz']

data = pd.read_csv(f"../data/ID_TO_NAME.csv")
all_id_to_name = {str(cat_id): data['name'][idx] for idx, cat_id in enumerate(data['id']) }

with open(f"../data/categories/Jewelry.json","r") as file: 
    jewelry_ids, jewelry_paths = json.load(file)

with open(f"../data/categories/Violin.json","r") as file: 
    violin_ids, violin_paths = json.load(file)

### Search Products

In [ ]:
cols = ['asin', 'price', 'title', 'rating', 'currency', 
        'is_prime', 'best_seller', 'price_upper', 
        'is_sponsored', 'manufacturer', 'sales_volume', 
        'pricing_count', 'reviews_count', 'is_amazons_choice', 
        'price_strikethrough', 'shipping_information','pos', 'rel_pos',
        'url_image']

sources = ['paid','organic','suggested','amazons_choices']

class Amazon_Search():
    def __init__(self,category_id,keywords,total_num_pages=7):
        self.domain = "com"
        self.category_id = category_id
        self.search_results = {} 
        for query in keywords: 
            self.search_results[query] = []
            progress = tqdm(range(total_num_pages))
            for page in progress:
                progress.set_description_str(f"Processing {self.category_id} {all_id_to_name[category_id]}: {query}") 
                results = self.search(query,page)
                if len(results['results']) == 0: 
                    break
                self.search_results[query].append(results['results'])
            self.save_data()  
        
    def search(self,query,page):
        task_params = {
            "target": "amazon_search",
            "query": query,
            "domain": self.domain, # USA 
            "page_from": page, # from 1 to 7 (already the most likely to be viewed by customers)
            "category": self.category_id, 
            "parse": True
        }

        username = os.environ.get('API_USERNAME')
        password = os.environ.get('API_PASSWORD')
        response = requests.post(
            'https://scraper-api.smartproxy.com/v2/scrape',
            json = task_params,
            auth = (username, password)
        )

        return response.json()

    def save_data(self):
        all_results = []
        def product(p,page,query,abs_pos,source):
            d = {}
            for c in cols: 
                if c in p: 
                    d[c] = p[c]
                else: 
                    d[c] = None
            d['source'] = source
            d['category_id'] = self.category_id
            d['keyword'] = query 
            d['page'] = page + 1
            d['position'] = abs_pos+1
            d['min_revenue'],d['discount'],d['discount_rate'] = 0,0,0
            if d['reviews_count'] is not None and d['price'] is not None: 
                d['min_revenue'] = d['reviews_count']*d['price']
            if d['price_strikethrough'] is not None: 
                d['discount'] = d['price_strikethrough'] - d['price']
                d['discount_rate'] = d['discount'] / d['price_strikethrough']
            else: 
                d['price_strikethrough'] = 0
            return d 

        for query, results in self.search_results.items(): 
            pages = [p[0]['content']['results'] for p in results]

            source_idx = {source:0 for source in sources}
            for pnum, page in enumerate(pages):
                if len(page['results']) == 0: continue
                for source in sources:
                    products = page['results'][source]
                    for p in products:
                        all_results.append(product(p,pnum,query,source_idx[source],source))
                        source_idx[source] += 1 
            df = pd.DataFrame(all_results)
            df.to_csv(f'../data/search_results/{self.category_id}_{all_id_to_name[self.category_id]}.csv', index=False)
                
        df = pd.DataFrame(all_results)
        df.to_csv(f'../data/search_results/{self.category_id}_{all_id_to_name[self.category_id]}.csv', index=False)


def process_results(ids, keywords,num_pages=7):
    for cat_id in ids:
        search_results = Amazon_Search(cat_id,keywords,num_pages)   

In [ ]:
searchterms = ['electronic cigarettes','e-cigarettes','smokeless inhaler','herbal cigarettes']
search_results = Amazon_Search("4078751",searchterms[0])

In [ ]:
process_results(["4078751"],searchterms[1:])

In [ ]:
search_results = process_results(violin_ids,['violin'])

In [43]:
search_results = process_results(jewelry_ids[11:12],materials[14:],num_pages=1)

Processing 3888121 Necklaces: rose quartz:   0%|          | 0/1 [00:00<?, ?it/s]


ConnectionError: HTTPSConnectionPool(host='scraper-api.smartproxy.com', port=443): Max retries exceeded with url: /v2/scrape (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x13b57d6d0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
search_results = process_results(jewelry_ids[12:],materials,num_pages=1)

In [ ]:
search_results = process_results(jewelry_ids[12:12],materials[14:],num_pages=1)

In [40]:
# Processing 3888121 Necklaces: rose quartz:   0%|          | 0/1 [00:05<?, ?it/s]

jewelry_ids.index("3888121"),len(jewelry_ids)

(11, 25)

In [41]:
materials.index("rose quartz")

14

In [ ]:
process_results(jewelry_ids,materials)

### Prices and approximate sales & revenue (if known)

In [ ]:
class Stats():
  def __init__(self,products,keyword=None):
      # Dictionary of the original entries in the aggregated CSV for each collection of search terms (i.e., keyword)
      if keyword is not None: 
          products = products[products.keyword == keyword]
      self.keyword = keyword
      self.category_id = products['category_id'][0] 
      products = products.to_dict(orient='records')
      self.products = {info['asin']: info for info in products}
      
      # Dictionary of dictionaries with central tendency statistics for each metric
      self.stats = {}

      # Dictionary mapping ASIN to a certain type of info
      self.prices = {}
      self.rating = {}
      self.reviews_count, self.sales = {}, {}
      self.best_sellers, self.prime, self.choice,self.sponsored = {},{},{},{}
      self.min_revenue_all = {}
      self.discounts, self.percentages = {}, {}
      for asin, info in self.products.items():
          self.prices[asin] = info['price']
          self.min_revenue_all[asin] = info['min_revenue']
          self.rating[asin] = info['rating']
          self.reviews_count[asin] = info['reviews_count']
          if info['price_strikethrough'] > 0:
              self.discounts[asin] = info['discount'] 
              self.percentages[asin] = info['discount_rate']
          if info['best_seller']: self.best_sellers[asin] = None
          if info['is_prime']: self.prime[asin] = None
          if info['is_sponsored']: self.sponsored[asin] = None
          if info['is_amazons_choice']: self.choice[asin] = None

          if isinstance(info['sales_volume'],str):
              if '+' not in info['sales_volume']: continue 
              self.pastmonth = info['sales_volume'].split("+")[0]
              if 'K' in self.pastmonth:
                self.pastmonth = 1000 * int(self.pastmonth.split("K")[0])
              self.sales[asin] = int(self.pastmonth)

      print(f'''There are {len(self.prices)} products with reviews:
            {len(self.sales)} with approximate sales volume for the past month,
            {len(self.best_sellers)} in Best Sellers,
            {len(self.choice)} in Amazon's Choice,
            {len(self.prime)} in Prime,
            {len(self.sponsored)} paid products in the search results, and
            {len(self.discounts)} products with price changes.''')
      
      self.all_stats('Organic')
      self.all_stats('Paid')

  def get_stats(self, key, dictionary):
    #   if key in self.stats:
        #   print(self.stats[key])
        #   return
      search_type, product_type, title = key.split(" - ")
      data = pd.Series(dictionary)
      mean = round(data.mean(),2)
      median = round(data.median(),2)
      if len(data.mode()) > 0:  
        mode = round(data.mode()[0],2)
        modefreq = data.value_counts()[data.mode()[0]]
        stdev = round(data.std(),2)
        max = round(data.max(),2)
        min = round(data.min(),2)
        percentiles = data.quantile([0.25, 0.75])
        self.stats[key] = {'Title': title, 'Category ID': self.category_id, 
                          'Keyword': self.keyword, 'Search Type': search_type, 
                          'Product Type':product_type, 'Mean': mean, 'Median': median, 
                          'Mode': mode, 'Mode Freq': modefreq, 'Std Dev': stdev,
                          'Max': max, 'Min': min,'Percentile25':round(percentiles[0.25],2),
                          'Percentile75':round(percentiles[0.75],2)}
    #   pprint(self.stats[key])

  def all_stats(self,category='Organic'):

      def get_relevant(dictionary,type='All'):
          min_rev, prices_relevant = {},{}
          ratings, reviews = {},{}
          discounts, percentages = {},{}
          for asin, item in dictionary.items():
              if category == 'Organic':
                if asin in self.sponsored: continue
              elif category == "Paid":
                if asin not in self.sponsored: continue
              min_rev[asin] = self.min_revenue_all[asin]
              prices_relevant[asin] = self.prices[asin]
              ratings[asin] = self.rating[asin]
              reviews[asin] = self.reviews_count[asin]
              if asin in self.discounts: 
                discounts[asin] = self.discounts[asin]
                percentages[asin] = self.percentages[asin]
          self.get_stats(f'{category} - {type} - Revenue',min_rev)
          self.get_stats(f'{category} - {type} - Price',prices_relevant)
          self.get_stats(f'{category} - {type} - Rating',ratings)
          self.get_stats(f'{category} - {type} - Reviews Count',reviews)
          self.get_stats(f'{category} - {type} - Amount Discounted',discounts)
          self.get_stats(f'{category} - {type} - Percent Discounted',percentages)

      get_relevant(self.reviews_count)
      get_relevant(self.best_sellers,"Best Seller")
      get_relevant(self.sales,"Past Month Sales Volume")
      get_relevant(self.prime,"Prime")
      get_relevant(self.choice,"Amazon's Choice")

In [ ]:
searches = os.listdir('../data/search_results')
stats = []
for file in searches:  
    products = pd.read_csv(f'../data/search_results/{file}')
    for keyword in set(products.keyword): 
        print(f"Processing {file}")
        product_stats = Stats(products,keyword)
        stats.extend(list(product_stats.stats.values()))
        print()
pd.DataFrame(stats).to_csv(f'../data/product_stats.csv', index=False)

In [ ]:
def logit_regression(products):
    data = {'price':[],'reviews_count':[],'min_revenue':[],'discount_rate':[],
            'choice':[],'prime':[],
            'sponsored':[],
            'high_sales':[]}
    for p in products:
        if p['reviews_count'] == 0 or p['price'] == 0: continue

        if p['is_sponsored']: data['sponsored'].append(1)
        else: data['sponsored'].append(0)

        if p['is_prime']: data['prime'].append(1)
        else: data['prime'].append(0)

        if p['is_amazons_choice']: data['choice'].append(1)
        else: data['choice'].append(0)

        if p['discount_rate'] > 0: data['discount_rate'].append(p['discount_rate'])
        else: data['discount_rate'].append(0.0)

        data['price'].append(p['price'])
        data['reviews_count'].append(p['reviews_count'])
        data['min_revenue'].append(p['min_revenue'])

        # dependent variable
        if p['best_seller'] == True or isinstance(p['sales_volume'],str): data['high_sales'].append(1)
        else: data['high_sales'].append(0)
    
    if sum(data['high_sales']) == 0: 
        return 
    
    df = pd.DataFrame(data)
    X = df.drop('high_sales', axis=1)
    y = df['high_sales']
    logit_model = sm.Logit(y, X).fit()
    print(logit_model.summary())
    return X, y

In [ ]:
searches = os.listdir('../data/search_results')
products = []
for file in searches:  
    data = pd.read_csv(f'../data/search_results/{file}')
    products.extend(data.to_dict('records'))

In [ ]:
df = pd.DataFrame(products)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

def combine_elements(series):
    return '; '.join(series.astype(str).unique())
combine_columns = ['position', 'keyword','category_id']
keep_columns = [col for col in df.columns if col not in combine_columns and col != 'asin']
PRODUCTS = df.groupby(['asin']).agg(
    {**{col: combine_elements for col in combine_columns},
     **{col: 'first' for col in keep_columns}}
).reset_index()

PRODUCTS = PRODUCTS.sort_values(by=["min_revenue","best_seller","rating","reviews_count"],ascending=[False,True,False,False])


def get_url(asin):
    return f"https://www.amazon.com/dp/{asin}"
PRODUCTS['url'] = PRODUCTS['asin'].apply(get_url)
def sales_vol(info):
    if info is None: return None
    num = info.split("bought in past month")[0]
    num = num.split("+")[0]
    if "K" in num: 
        num = num.split("K")[0] 
        return f"{int(num)*1000}+"
    return f"{num}+" 
PRODUCTS['sales_volume'] = PRODUCTS['sales_volume'].apply(sales_vol)

def round_values(value):
    return round(value,2)
PRODUCTS['discount_rate'] = PRODUCTS['discount_rate'].apply(round_values)
PRODUCTS['discount'] = PRODUCTS['discount'].apply(round_values)
PRODUCTS['min_revenue'] = PRODUCTS['min_revenue'].apply(round_values)


def set_to_zero(value):
    if pd.isna(value): return 0 
    else: return value 

PRODUCTS['discount'] = PRODUCTS['discount'].apply(set_to_zero)
PRODUCTS['discount_rate'] = PRODUCTS['discount_rate'].apply(set_to_zero)
PRODUCTS = PRODUCTS[PRODUCTS["min_revenue"] > 0.0]
PRODUCTS.to_csv('../data/products.csv')

In [ ]:
X, y = logit_regression(products)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
model = LogisticRegression() 
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
model.fit(x_train, y_train)
predictions = model.predict(x_test)

In [ ]:
# Logistic Regression with 3:1 train-test split
target_names = ['not high sales', 'high sales']
print(classification_report(y_test, predictions, target_names=target_names))